In [8]:
with open('Dataset/StudentHandbookDataset.txt', 'r', encoding='utf-8') as f:
    dataset = f.read()

print(f"📚 Dataset loaded: {len(dataset):,} characters")
print(f"📄 Estimated pages: ~{len(dataset) // 2000}")

📚 Dataset loaded: 171,284 characters
📄 Estimated pages: ~85


In [ ]:
import torch
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("\n🔧 Initializing embedding model...")
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",  # Best quality for academic text
    model_kwargs={
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'trust_remote_code': True
    },
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Embedding model loaded: all-mpnet-base-v2 (768 dimensions)")

In [ ]:
print("\n🔧 Setting up semantic chunker...")
text_splitter = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=80,  # Good balance for policy docs
    buffer_size=1,
    add_start_index=True  # Track position in original text
)


🔧 Setting up semantic chunker...


NameError: name 'SemanticChunker' is not defined

In [ ]:
# 3. Split raw text into semantic chunks using create_documents
print("\n📝 Creating semantic chunks from raw text...")
chunks = text_splitter.create_documents([dataset])
print(f"✅ Created {len(chunks)} semantic chunks")

# Analyze chunk quality
chunk_sizes = [len(chunk.page_content) for chunk in chunks]
print(f"\n📊 Chunk Analysis:")
print(f"   Average size: {np.mean(chunk_sizes):.0f} characters")
print(f"   Size range: {min(chunk_sizes)} - {max(chunk_sizes)} characters")
print(f"   Total chunks: {len(chunks)}")

# Show sample chunks
print("\n📋 Sample chunks:")
for i in range(min(3, len(chunks))):
    chunk_preview = chunks[i].page_content[:150].replace('\n', ' ')
    print(f"   Chunk {i+1}: {chunk_preview}...")


In [ ]:
print("\n🔄 Generating embeddings for all chunks...")
chunk_texts = [chunk.page_content for chunk in chunks]

# Process embeddings in batches to avoid memory issues
batch_size = 32
all_embeddings = []
for i in range(0, len(chunk_texts), batch_size):
    batch = chunk_texts[i:i+batch_size]
    batch_embeddings = embedding_model.embed_documents(batch)
    all_embeddings.extend(batch_embeddings)
    print(f"   Processed batch {i//batch_size + 1}/{(len(chunk_texts) + batch_size - 1)//batch_size}")

print(f"✅ Generated {len(all_embeddings)} embeddings")



In [ ]:

# 5. Build FAISS vector store for fast similarity search
print("\n🗄️ Building FAISS vector database...")
dimension = len(all_embeddings[0])
index = faiss.IndexFlatIP(dimension)  # Inner Product for cosine similarity

# Normalize embeddings for proper cosine similarity
embeddings_array = np.array(all_embeddings).astype('float32')
faiss.normalize_L2(embeddings_array)
index.add(embeddings_array)

print(f"✅ FAISS index ready: {index.ntotal:,} vectors ({dimension} dimensions)")


In [ ]:
# 6. Load high-quality language model for generation
print("\n🤖 Loading language model...")

# Configure 4-bit quantization for T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
from huggingface_hub import notebook_login

# This will prompt you to enter your HF token
notebook_login()


In [ ]:
# Use Mistral 7B for quality (perfect for T4)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

print(f"✅ Model loaded: {model_name}")
print(f"🎯 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
def retrieve_relevant_chunks(query, top_k=5):
    """Find most relevant chunks for the query"""
    # Embed the query
    query_embedding = embedding_model.embed_query(query)
    query_vector = np.array([query_embedding]).astype('float32')
    faiss.normalize_L2(query_vector)

    # Search FAISS index
    scores, indices = index.search(query_vector, top_k)

    # Return results with metadata
    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(chunks):  # Safety check
            chunk = chunks[idx]
            results.append({
                'text': chunk.page_content,
                'score': float(score),
                'chunk_id': int(idx),
                'start_pos': chunk.metadata.get('start_index', 0) if hasattr(chunk, 'metadata') else 0
            })

    return results

def generate_answer(query, context_chunks, max_new_tokens=350):
    """Generate answer using retrieved context"""
    # Combine context from relevant chunks
    context_parts = []
    for i, chunk in enumerate(context_chunks):
        context_parts.append(f"[Section {i+1}]\n{chunk['text']}")

    combined_context = "\n\n".join(context_parts)

    # Create optimized prompt for university handbook
    prompt = f"""<s>[INST] You are a university student advisor with access to the official student handbook. Answer the student's question accurately using only the provided handbook sections.

HANDBOOK SECTIONS:
{combined_context}

STUDENT QUESTION: {query}

Provide a clear, helpful answer based on the handbook information above. If the handbook doesn't contain enough information, say so. Be specific about policies, procedures, and requirements. [/INST]"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3072)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2,  # Low for factual accuracy
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Extract generated answer
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer_start = response.find("[/INST]") + len("[/INST]")
    answer = response[answer_start:].strip()

    return answer

def ask_handbook(question, top_k=5, show_sources=True):
    """Complete RAG pipeline for handbook queries"""
    print(f"\n❓ Question: {question}")
    print("=" * 70)

    # Retrieve relevant sections
    relevant_chunks = retrieve_relevant_chunks(question, top_k)

    if not relevant_chunks:
        print("❌ No relevant information found in handbook")
        return None

    if show_sources:
        print("📚 Found relevant handbook sections:")
        for i, chunk in enumerate(relevant_chunks):
            print(f"\n📄 Section {i+1} (Relevance: {chunk['score']:.3f})")
            preview = chunk['text'][:200].replace('\n', ' ')
            print(f"   {preview}...")

    # Generate answer
    print("\n🤔 Generating answer...")
    answer = generate_answer(question, relevant_chunks)

    print(f"\n💡 Answer:")
    print(answer)
    print("\n" + "=" * 70)

    return {
        'question': question,
        'answer': answer,
        'sources': relevant_chunks,
        'num_sources': len(relevant_chunks)
    }

# Test the system
print("\n\n🚀 RAG System Ready!")
print("Testing with university-specific questions...")

# Sample test questions for university handbook
test_questions = [
    "What are the graduation requirements?",
    "How do I withdraw from a course?",
    "What is the academic probation policy?",
    "What happens if I'm caught cheating?",
    "How do I change my major?"
]

# Run a test query
test_result = ask_handbook(test_questions[0])

# Interactive query function
def interactive_mode():
    """Interactive mode for asking questions"""
    print("\n🎓 Interactive University Handbook Assistant")
    print("Type 'quit' to exit")
    print("-" * 50)

    while True:
        question = input("\n❓ Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break

        if not question:
            print("Please enter a question!")
            continue

        try:
            ask_handbook(question)
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Uncomment to start interactive mode
# interactive_mode()

print("\n📋 System Summary:")
print(f"   📚 Processed: {len(chunks):,} semantic chunks")
print(f"   🔍 Embeddings: {len(all_embeddings):,} vectors")
print(f"   🤖 Model: {model_name}")
print(f"   💾 Ready for queries!")

In [ ]:
interactive_mode()